# Simple RNN TF1

** This is the implementation for TensorFlow 1.0 and python 3.4. It uses the Keras or the TF-RNN library for training.** For another more manual implementation see simple_rnn in dl_tutorial

In this notebook we consider a simple example of an RNN and used a quite artifical data generating process (if you have a better idea / story please contact me). 

The example has been motivated by:
http://r2rt.com/recurrent-neural-networks-in-tensorflow-i.html. 

Other Resources for RNNs:

* http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/
* http://r2rt.com/recurrent-neural-networks-in-tensorflow-i.html
* http://karpathy.github.io/2015/05/21/rnn-effectiveness/
* http://colah.github.io/posts/2015-08-Understanding-LSTMs/
* http://www.deeplearningbook.org/contents/rnn.html

In [1]:
from six.moves.cPickle import loads
import numpy as np
import sys
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(1)

import keras
%matplotlib inline
import matplotlib.pyplot as plt
tf.__version__, sys.version_info

Using TensorFlow backend.


('1.0.0',
 sys.version_info(major=3, minor=4, micro=3, releaselevel='final', serial=0))

In [2]:
# Global config variables (see below)
num_steps = 40     # number of truncated backprop steps
batch_size = 200  # number of minibatches b
num_classes_in = 3   # number of classes in the input
num_classes_out = 2   # number of classes in the output
state_size = 4    # number of classes in the state
learning_rate = 0.1

# Helper functions
def one_hot(Y, max):
    d = np.zeros((len(Y),max), dtype='int32')
    for row,col in enumerate(Y):
        d[row, col] = 1
    return d    

### Definition of the task

We consider a network which predicts at each point in time a variable $\hat{y}_t$ based on earlier values of $\hat{y}_{t'}$ covariates $x_t$. 

### Example data  (I screama, you screama, we all screama for I screama)

We need some data to play around with RNNs. They are capable of doing quite complicated things such as language models and so on. For this example, we want to generate the data ourself. We have to come up with a process which creates $x_t$ which itself can be influcenced by events $x_{t'}$ which happend before $t$. Further, we have to come up with $y_t$ which depends on $x_t'$ for timepoints $t' \le t$. 

To keep it simple, we analyse the following quite artifical process in which the weather $x_{t'}$ for $t' \le t$ influences our stock on icecream $y_t$. We then see if the RNN is capable of reconstructing that process.

#### Definition of the simple process
The weather $x_t$ at a certain point in time $t$ has three states (sunny, rainy, cloudy), which we model as $x_t = (1,0,0)$, $x_t = (0,1,0)$, and $x_t = (0,0,1)$ repectively. We assume that the weather is completly random (of course we could model more complex scenarios). 

We have an icecream store capable of holding 2 units of icecream and we start with a full store. When it is sunny we sell one unit of icecream. We have the strange policy that we order  on unit of icecream when it's cloudy. It takes 3 days to deliever the ice cream, we accept the ice cream if we do not have a full stock.

This enables us to model $y_t$ the state of the store $(1,0)$ for out of stock and $(0,1)$ for in stock. We create the one-hot-encoded data in the graph later. For now we use integers but keep in mind that the data is categorical. 

** The important part is that, we have values $y_t$ which can be prediced from earlier** $x_t$s

In [3]:
def gen_data(size=1000000):
    Xs = np.array(np.random.choice(3, size=(size,))) #Random Weather
    Y = []
    ice = 2 #Our stock of icecream at start
    for t,x in enumerate(Xs):
        # (t-3) >= 0 the first ice cream could be delivered on day 3
        # Xs[t - 3] claudy three days before today => we ordered ice cream
        # ice < 2 not full
        if (t - 3) >= 0 and Xs[t - 3] == 1 and ice < 2: 
            ice += 1
        if x == 0: # It is sunny we therefore sell ice, if we have
            if ice > 0: # We have ice cream
                ice -= 1
        if ice > 0: #We are not out of stock
            Y.append(1)
        else:
            Y.append(0)
    return Xs, np.array(Y)

In [4]:
X_train, Y_train = gen_data(50000) #Global variables holding the input and output
(X_train[0:50], Y_train[0:50])

(array([2, 0, 2, 2, 0, 0, 2, 1, 2, 2, 2, 2, 0, 2, 1, 0, 1, 1, 1, 1, 0, 0, 1,
        1, 0, 0, 0, 2, 2, 2, 1, 2, 1, 1, 2, 1, 2, 2, 0, 2, 0, 2, 2, 0, 0, 2,
        1, 0, 1, 1]),
 array([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1]))

### Forward pass in numpy
To better illustrate the used method, we first do a forward-pass of the RNN using numpy. We load the weights which we calculated previously with the cells below.

In [5]:
W_, b_, V_, bv_ = np.load('rnn_weights_tf1.npy')

### Architecture of the network 
We now define the network, we do not consider the output nodes yet.
A single RNN cell is shown in the figure below in the middle:

![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-SimpleRNN.png)
Image taken from: [Colah's RNN Blog](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

The joining of the two lines coming from the previous state $h_{t-1}$ and the current x-values $x_t$ is a concantination to a vector  $[h_{t-1}, x_{t}]$ of size `state_size + num_classes_in`. Alternatively, instead of concatinating, one could also use two matrices $W_x$ and $W_h$ and keep the states seperate. This is mathematically completely identical. The new state $h_t$ is then calculated as:

$$
    h_{t} = \tanh([h_{t-1}, x_{t}] \cdot W + b) = \tanh(h_{t-1} \cdot W_h + x_{t} \cdot U + b)
$$

The dynamic of the hidden state $h_{t}$ is determined by $W$ (and $b$):

In [6]:
W_, b_

(array([[-0.17254561,  1.21135914,  0.0326971 , -0.84142756],
        [-1.13171566, -0.37227634,  0.72233802,  1.36369407],
        [-0.19222833, -0.18928526,  0.67648065, -0.53148365],
        [ 1.0540185 ,  0.62964875, -0.89659381,  0.37798104],
        [-0.15900061,  1.03748   , -0.86709827, -0.44786388],
        [ 0.1902504 , -1.23239243,  0.05057732,  0.41327468],
        [-1.56150496,  0.3125982 , -1.91797554, -1.17750788]], dtype=float32),
 array([ 0.11968273,  0.29767367,  0.19570114, -0.35632327], dtype=float32))

In [7]:
# The first state
h0 = np.zeros(state_size) #We start with 0 initial state
x1 = one_hot(X_train, num_classes_in)[0] #Make a vector

#<---- your code here (calculate the hidden state h1) ---->
h1 = np.tanh(np.matmul(np.concatenate([x1, h0]), W_) + b_)
#<---- end your code here ---->

print(h0, "--->", h1)

[ 0.  0.  0.  0.] ---> [-0.0724186   0.10796594  0.70248095 -0.71030884]


We could repeat those transitions of the hidden states to get a sequence of hidden states:

$h_0 \rightarrow h_1 \rightarrow h_2 \rightarrow h_3 \rightarrow h_4 \ldots $

In [8]:
def rnn_forward(state, X_train):
    hs = []
    for t in range(len(X_train)):
        # Note that TF concatenates [Input, State]
        state = np.tanh(np.matmul(np.concatenate([X_train[t,:],state]), W_) + b_)
        hs.append(state)
    return hs

In [9]:
rnn_forward(h0, one_hot(X_train[0:5],num_classes_in))

[array([-0.0724186 ,  0.10796594,  0.70248095, -0.71030884]),
 array([ 0.79921588,  0.45236803,  0.92130586, -0.14572111]),
 array([ 0.80076431, -0.09969379,  0.08921116, -0.23169287]),
 array([ 0.82301409,  0.31563207,  0.59769869, -0.22678265]),
 array([ 0.84323308,  0.91332454, -0.30769879, -0.47303671])]

We add some output. For each time step the output is produced by multiplying the hidden state with:

$o_t = h_t \cdot V + b_{\tt{v}}$

This is a logit, the final the probability of output class is the softmax of the logit.

In [10]:
#<---- your code here (calculate the output state o1 for timestep 1 from h1, V_ and the bias bv_) ---->
o1 = np.matmul(h1, V_) + bv_
#<---- your code here (calculate probability from the state o1) using softmax ---->
prob_1 = np.exp(o1)/np.sum(np.exp(o1))
#<---- end your code here  ---->
o1, prob_1

(array([-0.22730275,  0.63843659]), array([ 0.29614163,  0.70385837]))

In [11]:
h = rnn_forward(h0, one_hot(X_train,3))
pt = []
for t in range(len(h)):
    ot = np.matmul(h[t], V_) + bv_
    pt.append(np.exp(ot)/np.sum(np.exp(ot)))

In [12]:
pt[0:10], np.argmax(pt[0:30],axis=1), Y_train[0:30]

([array([ 0.29614163,  0.70385837]),
  array([ 0.7524353,  0.2475647]),
  array([ 0.17888804,  0.82111196]),
  array([ 0.61858992,  0.38141008]),
  array([ 0.98360421,  0.01639579]),
  array([ 0.98868512,  0.01131488]),
  array([ 0.98215238,  0.01784762]),
  array([ 0.97015616,  0.02984384]),
  array([ 0.98632135,  0.01367865]),
  array([ 0.96048924,  0.03951076])],
 array([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1]),
 array([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1]))

In [13]:
np.average(np.argmax(pt, axis=1) == Y_train)

0.98782000000000003

In [14]:
tot_loss = 0
for i in range(len(Y_train)):
    tot_loss += -np.log(pt[i][Y_train[i]])

In [15]:
tot_loss / len(Y_train)

0.061891539492980138

## Training of the network
### Preparation of the Minibatch

In this example, we have in principle a large stream of data $x$ and $y$. For efficiency reason we split the stream in minibatches of a certain length. For this task we could also imagin to have several realizations of that icecream process, so that it would also be natural to split the process into mini batches. 

For simplicity, we create the minibatch by randomly cutting out `batch_size` entries of fixed length `num_steps`. Other, more advanced ways of doing so are possible. See e.g. https://danijar.com/variable-sequence-lengths-in-tensorflow/. For the time being, we thus consider the input tensor $X_{btc}$ for the minibatch to be of the following form:

* $b$ having `batch_size` entries
* $t$ loops over the unrolled timestamps (`num_steps`)
* $c$ has the dimension of the one-hot-coded input classes 

In [16]:
def data_matrix(Xs, Ys, size = 32, num_steps = 50):
    data_x = np.zeros([size, num_steps, 3], dtype=np.int32)
    data_y = np.zeros([size, num_steps, 2], dtype=np.int32)
    for i in range(1,size):
        s = int(np.random.uniform(0, len(Xs) - num_steps))
        data_x[i] = one_hot(Xs[s : s + num_steps],3)
        data_y[i] = one_hot(Ys[s : s + num_steps],2)
    return data_x, data_y

In [17]:
X, Y = data_matrix(X_train, Y_train, size=10000, num_steps=num_steps)
X.shape, Y.shape
#print (X[0:2,0:5])
#print (Y[0:2,0:5])

((10000, 40, 3), (10000, 40, 2))

## Building the model with keras

**Keras assumes (batch, time, input_dimension)** as input tensor. The batch dimension needs not to be specified (side note that specifying it to `None` would not work).


In our model we calculate the loss using all hidden timepoints. This is many-to-many situation is different for example to a sentiment classifier, where we we have a many-to-one situation. 

Not just the latest one (in time). Hence we set `return_sequences=True`.

In [18]:
model = keras.models.Sequential()
rnn = keras.layers.SimpleRNN(state_size, input_shape=(num_steps, 3), return_sequences=True, name='RNN')
model.add(rnn) 
#model.input_shape --> (None, 40, 3)
#model.output_shape --> (None, 40, 4) 

# <-- Your code here 
# Add an output layer connecting with the hidden state of the RNN
model.add(keras.layers.Dense(2))
# <-- End code here 

#model.output_shape --> (None, 40, 2) 
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [19]:
model.fit(X, Y, epochs=100, verbose=2, validation_split=0.05)

Train on 9500 samples, validate on 500 samples
Epoch 1/100
1s - loss: 4.4168 - acc: 0.4192 - val_loss: 3.3592 - val_acc: 0.4812
Epoch 2/100
1s - loss: 1.8418 - acc: 0.5007 - val_loss: 0.7942 - val_acc: 0.5287
Epoch 3/100
1s - loss: 0.6630 - acc: 0.6107 - val_loss: 0.6195 - val_acc: 0.6765
Epoch 4/100
1s - loss: 0.5912 - acc: 0.6933 - val_loss: 0.5516 - val_acc: 0.7248
Epoch 5/100
1s - loss: 0.4634 - acc: 0.7863 - val_loss: 0.4031 - val_acc: 0.8275
Epoch 6/100
1s - loss: 0.3749 - acc: 0.8426 - val_loss: 0.3633 - val_acc: 0.8565
Epoch 7/100
1s - loss: 0.3404 - acc: 0.8611 - val_loss: 0.3390 - val_acc: 0.8568
Epoch 8/100
1s - loss: 0.3168 - acc: 0.8463 - val_loss: 0.3152 - val_acc: 0.8331
Epoch 9/100
1s - loss: 0.2948 - acc: 0.8175 - val_loss: 0.2945 - val_acc: 0.8074
Epoch 10/100
2s - loss: 0.2764 - acc: 0.7833 - val_loss: 0.2823 - val_acc: 0.7616
Epoch 11/100
1s - loss: 0.2629 - acc: 0.7454 - val_loss: 0.2717 - val_acc: 0.7335
Epoch 12/100
1s - loss: 0.2506 - acc: 0.7276 - val_loss: 0.2

In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
RNN (SimpleRNN)              (None, 40, 4)             32        
_________________________________________________________________
dense_1 (Dense)              (None, 40, 2)             10        
Total params: 42.0
Trainable params: 42
Non-trainable params: 0.0
_________________________________________________________________


In [21]:
# We use the training, since we did use only a random subset of the data
X, Y = data_matrix(X_train, Y_train, size=1000, num_steps=num_steps)
model.evaluate(X,Y)

 800/1000 [=======================>......] - ETA: 0s

[0.20355686962604522, 0.64475000476837163]

In [22]:
rnn.get_weights()

[array([[-1.23313642, -0.58405763,  0.33195788, -0.13621248],
        [ 0.18067464,  1.42137611, -0.06325942,  0.17167008],
        [ 0.15937355, -0.62480474,  0.33110496,  0.21680324]], dtype=float32),
 array([[ 1.04436243, -0.00809373,  0.03049999,  0.11092529],
        [ 0.09025644,  0.28329077,  0.56369871,  0.62143368],
        [ 0.34117034, -0.39231658, -0.50228685,  2.75797248],
        [ 1.54123604, -0.056024  ,  0.06839988,  0.76748008]], dtype=float32),
 array([-0.397686  ,  0.08730865,  0.09118398,  0.04391335], dtype=float32)]

## Using the TensorFlow API

Alternatively one can use the TensorFlow-API for creating RNNs. In principle there are two TensorFlow methods. The first, kind of deprecated one, builds a graph from the unrolled network. This API has issues in performance, first of all the creation of the graph takes quite some time. Further, and this is a bit it is also slower during runtime. Therefore, the novel dynamic API should be prefered. If you want to use sequences of variable length see:  https://danijar.com/variable-sequence-lengths-in-tensorflow/

In [23]:
def get_batch(Xs, Ys, batch_size = 32, num_steps = 50):
    data_x = np.zeros([batch_size, num_steps], dtype=np.int32)
    data_y = np.zeros([batch_size, num_steps], dtype=np.int32)
    for i in range(1,batch_size):
        s = int(np.random.uniform(0, len(Xs) - num_steps))
        data_x[i] = Xs[s : s + num_steps]
        data_y[i] = Ys[s : s + num_steps]  
    return data_x, data_y

In [24]:
tf.reset_default_graph()
tf.set_random_seed(42)
# Placeholders
x = tf.placeholder(tf.int32, [batch_size, num_steps], name='input_placeholder')
y = tf.placeholder(tf.int32, [batch_size, num_steps], name='labels_placeholder')
init_state = tf.zeros([batch_size, state_size])

# RNN Inputs
# One hot encoding.
x_one_hot = tf.one_hot(x, num_classes_in)
# We want the following dimensions [batch_size, Max_Length, num_classes_in]
rnn_inputs = tf.transpose(x_one_hot, perm=(0,1,2))
rnn_inputs

<tf.Tensor 'transpose:0' shape=(200, 40, 3) dtype=float32>

### Definition of the basic cell

We have to define the elementary cell, which has a state of a given size. As above we use the basic RNN-Cell which is described in: https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/rnn/python/ops/core_rnn_cell_impl.py

In [25]:
#cell = tf.nn.rnn_cell.BasicRNNCell(state_size) 
#cell = tf.nn.rnn_cell.BasicLSTMCell(state_size)
# For tf1.0 the cells have been temporarily moved to a contib see: https://github.com/tensorflow/models/issues/919
cell = tf.contrib.rnn.BasicRNNCell(state_size)
init_state = cell.zero_state(batch_size, tf.float32)
rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, rnn_inputs, initial_state=init_state)

In [26]:
rnn_outputs

<tf.Tensor 'rnn/transpose:0' shape=(200, 40, 4) dtype=float32>

The output $o_{btk}$ tensor produces for each minibatch and timepoint a 4 (number of output states) dimensional vector indexed by $k$. For each timepoint and batch, we later want to compare this with the corresponding y-value with has the shape $y_{bt}$. In a first step we flatten the b and t dimension to a $200*40 = 8000$ dimensional vector.

In [27]:
#reshape rnn_outputs and y so we can get the logits in a single matmul
rnn_outputs = tf.reshape(rnn_outputs, [-1, state_size])
y_reshaped = tf.reshape(y, [-1])
y_reshaped, rnn_outputs

(<tf.Tensor 'Reshape_1:0' shape=(8000,) dtype=int32>,
 <tf.Tensor 'Reshape:0' shape=(8000, 4) dtype=float32>)

In [28]:
with tf.variable_scope('softmax'):
    V = tf.get_variable('V', [state_size, num_classes_out])
    bv = tf.get_variable('bv', [num_classes_out], initializer=tf.constant_initializer(0.0))

logits = tf.matmul(rnn_outputs, V) + bv
logits, y_reshaped

(<tf.Tensor 'add:0' shape=(8000, 2) dtype=float32>,
 <tf.Tensor 'Reshape_1:0' shape=(8000,) dtype=int32>)

In [29]:
#writer = tf.summary.FileWriter("tb_simple_rnn_tf1/dd", tf.get_default_graph()) 
#writer.close()
#!tensorboard --logdir=tb_simple_rnn_tf1/

In [30]:
total_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_reshaped, logits=logits))
train_step = tf.train.AdamOptimizer(learning_rate).minimize(total_loss)

In [31]:
Y = None
X = None
count = 0
sum_tr_losses = 0
sess = tf.Session()
#with tf.Session() as sess:
sess.run(tf.initialize_all_variables())
for i in range(1000):
    X, Y = get_batch(X_train, Y_train, batch_size, num_steps)
    tr_losses, _ = sess.run([total_loss, train_step], feed_dict={x:X, y:Y})
    count += 1
    sum_tr_losses += tr_losses
    if (i < 10) or (i % 200 == 0):
        print ("{} {}".format(i, sum_tr_losses / count))
        count = 0
        sum_tr_losses = 0

Instructions for updating:
Use `tf.global_variables_initializer` instead.
0 0.6798593997955322
1 0.6539711952209473
2 0.632749617099762
3 0.6084426045417786
4 0.5754246115684509
5 0.5737156271934509
6 0.5571834444999695
7 0.522765040397644
8 0.4930891692638397
9 0.4334166347980499
200 0.2222905765965347
400 0.19082847386598586
600 0.17908946476876736
800 0.16760613091289997


### Test on the training set

In [32]:
X, Y = get_batch(X_train, Y_train, batch_size, num_steps)
loss_train = sess.run(total_loss, feed_dict={x:X, y:Y})
loss_train

0.15980929

### Getting the relevant weights

In [33]:
# Finding the relavant weights
#ops = tf.get_default_graph().get_operations()
#for i in ops:
#   print(i.name)

In [34]:
graph = tf.get_default_graph()
W = graph.get_tensor_by_name('rnn/basic_rnn_cell/weights:0')
b = graph.get_tensor_by_name('rnn/basic_rnn_cell/biases:0')
W_,b_,V_,bv_ = sess.run([W,b,V,bv])
print (W_,b_,V_,bv_)
np.save('rnn_weights_tf1', [W_,b_,V_,bv_])

[[-0.19089293  1.2079829   0.00908419 -0.91102362]
 [-1.07885635 -0.34288317  0.72712117  1.41788387]
 [-0.20597383 -0.19254826  0.7025221  -0.54543966]
 [ 1.03414631  0.67596245 -0.93610775  0.37894768]
 [-0.16711316  1.02640092 -0.89726377 -0.42267284]
 [ 0.20204762 -1.21461511  0.02345279  0.41776735]
 [-1.44821215  0.28976035 -1.87600672 -1.1563133 ]] [ 0.1347418   0.28151226  0.17300364 -0.32107899] [[-0.18110763  0.14144787]
 [ 2.24330831 -3.24425101]
 [-0.05769794  0.35383844]
 [ 0.0802527  -0.5142712 ]] [-0.35922614  0.35923496]
